# Update libraries

In [0]:
!pip install --upgrade torchvision

# Install package

In [0]:
!pip install --force-reinstall $root/Sketches-0.0.1-py3-none-any.whl

# Set paths

In [0]:
import os

from utils.utils import Params

In [0]:
root = "/dbfs/sketches"
model_dir = "/experiments/base_model"

In [0]:
%load_ext tensorboard

# Check configs

In [0]:
param_path = os.path.join(root+model_dir, "params.yml")
params = Params(param_path)
print(params.__dict__)

In [0]:
params.batch_size = 64
#params.learning_rate = 0.01
#params.decay = 0.0005

print(params.__dict__)

In [0]:
params.save(param_path)

# Train model

In [0]:
!python $root/train.py --data_dir $root --model_dir $root$model_dir

In [0]:
!cat $root$model_dir/train.log

# Visualize

In [0]:
%tensorboard --logdir $root$model_dir/runs --port 6009

In [0]:
!cat $root$model_dir/metrics_val_last.yml

In [0]:
!cat $root$model_dir/metrics_val_best.yml